In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [9]:
# SETUP EVALUATORS
from Evaluation.Evaluator import EvaluatorHoldout

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2629 (20.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2189 (17.3%) Users that have less than 1 test interactions


In [2]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender, MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender


MAP_recommender_per_group = {}

collaborative_recommender_class = {"UserKNNCF": UserKNNCFRecommender,
                                   "ItemKNNCF": ItemKNNCFRecommender,
                                   "P3alpha": P3alphaRecommender,
                                   "RP3beta": RP3betaRecommender,
                                   "SLIM_ELASTIC": MultiThreadSLIM_SLIMElasticNetRecommender,
                                   "EASE_R": EASE_R_Recommender,
                                   }
collaborative_recommender_class.items()

recommender_object_dict = {}

for label, recommender_class in collaborative_recommender_class.items():
    recommender_object = recommender_class(URM_train_validation)
    recommender_object_dict[label] = recommender_object
    
    
recommender_object_dict["SLIM_ELASTIC"].fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668, workers=7)
recommender_object_dict["P3alpha"].fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
recommender_object_dict["RP3beta"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
recommender_object_dict["UserKNNCF"].fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                         feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
recommender_object_dict["ItemKNNCF"].fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
recommender_object_dict["EASE_R"].fit(topK=24, l2_norm=37.54323189430143)

UserKNNCFRecommender: URM Detected 211 ( 1.7%) users with no interactions.
UserKNNCFRecommender: URM Detected 118 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 211 ( 1.7%) users with no interactions.
ItemKNNCFRecommender: URM Detected 118 ( 0.5%) items with no interactions.
P3alphaRecommender: URM Detected 211 ( 1.7%) users with no interactions.
P3alphaRecommender: URM Detected 118 ( 0.5%) items with no interactions.
RP3betaRecommender: URM Detected 211 ( 1.7%) users with no interactions.
RP3betaRecommender: URM Detected 118 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: URM Detected 211 ( 1.7%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 118 ( 0.5%) items with no interactions.
EASE_R_Recommender: URM Detected 211 ( 1.7%) users with no interactions.
EASE_R_Recommender: URM Detected 118 ( 0.5%) items with no interactions.


100%|█████████▉| 22216/22222 [00:46<00:00, 482.78it/s]


P3alphaRecommender: Similarity column 22222 (100.0%), 7454.50 column/sec. Elapsed time 2.98 sec
RP3betaRecommender: Similarity column 22222 (100.0%), 6074.14 column/sec. Elapsed time 3.66 sec
Similarity column 12638 (100.0%), 10126.68 column/sec. Elapsed time 1.25 sec
Similarity column 22222 (100.0%), 10158.70 column/sec. Elapsed time 2.19 sec
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 1.56 min


In [10]:
from xgboost import XGBRanker
from Recommenders.XGBoostTrain import XGBoostTrain
import optuna as op
import xgboost as xgb

def objective(trial):
    param = {
        "verbosity": 0,
        "objective": "rank:pairwise",
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-8, 1.0, log=True),
        "eval_metric": "auc",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 1.0, log=True),
    }

    if param["booster"] == "gbtree" or param["booster"] == "dart":
        param["max_depth"] = trial.suggest_int("max_depth", 1, 9)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)
    xgb_model = XGBRanker(**param)
    # XGBRanker(
    #     objective='rank:pairwise',
    #     n_estimators=1000,
    #     learning_rate=0.19823429576094637,
    #     reg_alpha=47,
    #     reg_lambda=0.313,
    #     max_depth=3,
    #     verbosity=0,
    #     booster="gbtree",
    #     colsample_bytree=0.6113704247857885,
    #     gamma=8.964184693722684,
    #     min_child_weight=7.0,
    #     tree_method="hist"
    # )
    model = XGBoostTrain(URM_train_validation, URM_val=URM_test, recommenders=recommender_object_dict, XGB_model=xgb_model)
    # Add a callback for pruning.
    #pruning_callback = op.integration.XGBoostPruningCallback(trial, "validation-auc")
    model.fit()
        
    result, _ = evaluator_test.evaluateRecommender(model)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    
}

study_name = "XGB-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize",pruner=op.pruners.MedianPruner(n_warmup_steps=5), load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=10)

[I 2023-12-05 18:07:55,118] Using an existing study with name 'XGB-study' instead of creating a new one.


XGBoostTrain: URM Detected 211 ( 1.7%) users with no interactions.
XGBoostTrain: URM Detected 118 ( 0.5%) items with no interactions.
CREATING THE DATAFRAME


100%|██████████| 12638/12638 [00:04<00:00, 2974.90it/s]


POPULATING THE DATAFRAME


100%|██████████| 12638/12638 [00:19<00:00, 653.81it/s]


...

100%|██████████| 1000/1000 [00:00<00:00, 4301.33it/s]


CREATING THE DATAFRAME


100%|██████████| 1000/1000 [00:00<00:00, 2696.70it/s]


POPULATING THE DATAFRAME


 26%|██▌       | 260/1000 [00:00<00:01, 621.47it/s]

In [16]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2184 (17.3%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10454 (100.0%) in 8.71 sec. Users per second: 1200


(       PRECISION PRECISION_RECALL_MIN_DEN   RECALL      MAP MAP_MIN_DEN  \
 cutoff                                                                    
 10      0.089028                 0.150759  0.12639  0.04558    0.076747   
 
             MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.26035  0.140004  0.104469  0.493495      0.340601  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827188          0.408213    0.827188       0.057399   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.156523                   0.997471             0.164554   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                            

In [17]:
#final.fit(alphas=[3,1.25])
evaluator_test.evaluateRecommender(slim)

EvaluatorHoldout: Processed 10454 (100.0%) in 6.85 sec. Users per second: 1526


(       PRECISION PRECISION_RECALL_MIN_DEN   RECALL      MAP MAP_MIN_DEN  \
 cutoff                                                                    
 10      0.079759                 0.137157  0.11639  0.03848    0.066675   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.234882  0.125152  0.094654  0.475703      0.298827  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.827188          0.393496    0.827188       0.050161   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.020847                   0.997362             0.145638   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                         